# L1 MDP
---
# 2.1 鸳鸯系统马尔可夫决策过程
左上为雄鸟，右上为雌鸟，中间有两道障碍物。目标：雄鸟找到雌鸟。

https://blog.csdn.net/qq_43058281/article/details/114886903

# 五、代码实现 （python)

"""

In [4]:
import pygame
import math
import time
import random
import numpy as np

class YuanYangEnv:
    # 初始化子函数
    def __init__(self):
        # 状态空间
        self.states = []# 0-99
        for i in range(0,100):
            self.states.append(i)
        self.actions=['e','w','n','s']
        self.gamma = 0.8
        # 值函数
        self.value = np.zeros((10,10))# 10*10的表格

        # 设置渲染属性
        self.viewer = None # 一个渲染窗口
        # 帧速率是指程序每秒在屏幕山绘制图像的数目，我们可以用FPS来表示它。一般的计算机都能达到每秒60帧的速度。如果我们把帧速率讲得比较低，那么游戏也会看上去较为卡顿。
        self.FPSCLOCK = pygame.time.Clock()
        # 屏幕大小
        self.screen_size = (1200,900)
        # 雄鸟当前位置
        self.bird_position = (0,0)
        # 雄鸟在x方向每走一次像素为120
        self.limit_distance_x = 120
        # 雄鸟在y方向每走一次像素为90
        self.limit_distance_y = 90
        # 每个障碍物大小为120像素*90像素
        self.obstacle_size = [120,90]
        # 一共有两个障碍物墙，每个障碍物墙由8个小障碍物组成
        self.obstacle1_x = []
        self.obstacle1_y = []
        self.obstacle2_x = []
        self.obstacle2_y = []
        for i in range(8):
            # 第一个障碍物
            self.obstacle1_x.append(360)
            if i <= 3:
                self.obstacle1_y.append(90*i)
            else:
                self.obstacle1_y.append(90*(i+2))
            # 第二个障碍物
            self.obstacle2_x.append(720)
            if i <= 4:
                self.obstacle2_y.append(90*i)
            else:
                self.obstacle2_y.append(90*(i+2))
        # 雄鸟初始位置
        self.bird_male_init_position = [0,0]
        # 雄鸟当前位置
        self.bird_male_position = [0,0]
        # 雌鸟初始位置
        self.bird_female_init_position = [1080,0]

    # 雄鸟碰撞检测子函数
    def collide(self, state_position):
        # 用标志flag,flag1,flag2分别表示是否与障碍物、障碍物墙1、障碍物墙2发生碰撞
        flag = 1
        flag1 = 1
        flag2 = 1
        # 检测雄鸟是否与第一个障碍物墙发生碰撞
        # 找到雄鸟与第一个障碍物所有障碍物x方向和y方向最近的障碍物的坐标差
        # 并判断最近的坐标差是否大于一个最小运动距离
        # 如果大于等于 就不会发生碰撞
        dx = []
        dy = []
        for i in range(8):
            dx1 = abs(self.obstacle1_x[i] - state_position[0])
            dx.append(dx1)
            dy1 = abs(self.obstacle1_y[i] - state_position[1])
            dy.append(dy1)
        mindx = min(dx)
        mindy = min(dy)
        if mindx >= self.limit_distance_x or mindy >= self.limit_distance_y:
            flag1 = 0  # 没碰
        # 是否与第二个障碍物墙碰撞
        second_dx = []
        second_dy = []
        for i in range(8):
            dx2 = abs(self.obstacle2_x[i] - state_position[0])
            second_dx.append(dx2)
            dy2 = abs(self.obstacle2_y[i] - state_position[1])
            second_dy.append(dy2)
        mindx = min(second_dx)
        mindy = min(second_dy)
        if mindx >= self.limit_distance_x or mindy >= self.limit_distance_y:
            flag2 = 0  # 没碰
        if flag1 == 0 and flag2 == 0:
            flag = 0  # 没碰
        # 是否超出边界，如果是，也认为发生碰撞
        if state_position[0] > 1080 or state_position[0] < 0 or state_position[1] > 810 or state_position[1] < 0:
            flag = 1  # 碰了
        # 返回碰撞标志位
        return flag

    # 雄鸟是否找到雌鸟子函数
    def find(self,state_position):
        # 设置标志位flag
        # 判断雄鸟当前位置和雌鸟位置坐标差，雄安与最小运动距离则为找到
        flag = 0
        if abs(state_position[0] - self.bird_female_init_position[0]) < self.limit_distance_x and abs(
                state_position[1] - self.bird_female_init_position[1]) < self.limit_distance_y:
            flag = 1
        return flag

    # 状态转化为像素坐标子函数
    def state_to_position(self, state):
        i = int(state / 10)
        j = state % 10
        position = [0, 0]
        position[0] = 120 * j
        position[1] = 90 * i
        return position

    # 像素转化为状态坐标子函数
    def position_to_state(self, position):
        i = position[0] / 120
        j = position[1] / 90
        return int(i * 10 + j)

    # 重置环境子函数
    def reset(self):
        # 随机产生一个合法的初始位置，不能在障碍物处，不能在雌鸟处，
        flag1 = 1
        flag2 = 1
        while flag1 or flag2 == 1:
            # 随机产生初始状态0-99，random.random()产生一个0-1的随机数
            state = self.states[int(random.random()*len(self.states))]
            state_position = self.state_to_position(state)
            flag1 = self.collide(state_position)
            flag2 = self.find(state_position)
        return state


    # 状态转移子函数
    # 首先判断当前坐标是否与障碍物碰撞或是否是终点，如果是，结束转换
    # 如果否，更具运动学进行像素位置坐标的转换
    # 最后判断碰撞和是否找到
    # 没找到：回报为0；找到：回报为1；碰撞：回报为-1
    def transform(self,state,action):
        # 将当前状态转化为坐标
        current_position = self.state_to_position(state)
        next_position = [0,0]
        flag_collide = 0
        flag_find = 0
        # 判断是否与障碍物碰撞
        flag_collide = self.collide(current_position)
        # 判断是否为终点
        flag_find = self.find(current_position)
        if flag_collide==1 or flag_find ==1:
            return state,0,True # 结束
        # 状态转移
        if action == 'e':
            next_position[0]=current_position[0] + 120
            next_position[1]=current_position[1]
        if action == 'w':
            next_position[0] = current_position[0] - 120
            next_position[1] = current_position[1]
        if action == 'e':
            next_position[0] = current_position[0] + 120
            next_position[1] = current_position[1]
        if action == 'n':
            next_position[0] = current_position[0]
            next_position[1] = current_position[1] - 90
        if action == 's':
            next_position[0] = current_position[0]
            next_position[1] = current_position[1] + 90
        # 判断是否碰撞
        flag_collide = self.collide(next_position)
        # 如果碰撞，回报为-1，并结束
        if flag_collide==1:
            return self.position_to_state(current_position),-1,True # 结束
        # 判断是否为终点
        flag_find = self.find(next_position)
        if flag_find == 1:
            return self.position_to_state(next_position),1,True # 结束
        return self.position_to_state(next_position),0,False # 不结束

    # 游戏结束控制子函数
    # 调用pygame中的事件用来结束游戏，如果结束，则pygame会结束渲染
    def gameover(self):
        for event in pygame.event.get():
            if event.type == QUIT:
                exit()

    # 游戏渲染子函数
    def render(self):
        # 首先判断环境是否有一个游戏窗口，如果没有，调用pygame.display.set_mode设置一个游戏窗口
        # 调用load_bird_male(),load_bird_female(),load_background(),load_obstacle()函数下载游戏环境需要的突破 这四个函数的实现在load.py中进行
        # 下载完后，利用pygame自带的blit函数将图片画到窗口上
        if self.viewer is None:
            pygame.init()

            # 画一个窗口
            self.viewer = pygame.display.set_mode(self.screen_size,0,32)# 32表示深度
            pygame.display.set_caption("yuanyang")
            # 下载图片
            self.bird_male = load_bird_male()
            self.bird_female = load_bird_female()
            self.background = load_background()
            self.obstacle = load_obstacle()
            # self.viewer.blit(self.bird_male,self.bird_male_init_position)
            # 在幕布上画图片
            self.viewer.blit(self.bird_female,self.bird_female_init_position)
            self.viewer.blit(self.background,(0,0))
            self.font = pygame.font.SysFont('times',15)# 字体大小15

        # 如果环境中已经有窗口
        # 则依次画：背景图，11条纵向直线，11条横向直线，第一个障碍物墙，第二个障碍物墙，雄鸟，值函数
        # 画完后，调用pygame.display.update将这些原色更新到幕布中
        # 调用子函数gameover()来检查是否要结束有游戏
        self.viewer.blit(self.background,(0,0))
        # 画直线
        for i in range(11):
            pygame.draw.lines(self.viewer,(255,255,255),True,((120*i,0),(120*i,900)),1)
            pygame.draw.lines(self.viewer, (255, 255, 255), True, ((0, 90*i), (1200, 90*i)), 1)
        self.viewer.blit(self.bird_female,self.bird_female_init_position)
        # 画障碍物
        for i in range(8):
            self.viewer.blit(self.obstacle, (self.obstacle1_x[i], self.obstacle1_y[i]))
            self.viewer.blit(self.obstacle, (self.obstacle2_x[i], self.obstacle2_y[i]))

        # 画雄鸟
        self.viewer.blit(self.bird_male,self.bird_male_position)
        # 画值函数
        for i in range(10):
            for j in range(10):
                surface = self.font.render(str(round(float(self.value[i,j]),3)),True,(0,0,0))
                self.viewer.blit(surface,(120*i+5,90*j+70))
        pygame.display.update()
        self.gameover()
        self.FPSCLOCK.tick(30)

# 主函数程序测试
# 首先声明一个鸳鸯环境的示例yy
# 然后调用渲染函数将鸳鸯系统绘制出来
# 最后调用循环语句检查是都要结束游戏渲染
if __name__ == "__main__":
    yy = YuanYangEnv()
    yy.render()
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                exit()

SystemExit: 

D:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import pygame
import os
from pygame.locals import *
# from sys import  exit
# #得到当前工程目录
# current_dir = os.path.split(os.path.realpath(__file__))[0]
# print(current_dir)
#得到文件名
bird_file = r"bird.png"
obstacle_file = r"obstacle.png"
background_file = r"background.png"
#创建小鸟，
def load_bird_male():
    bird = pygame.image.load(bird_file).convert_alpha()
    return bird
def load_bird_female():
    bird = pygame.image.load(bird_file).convert_alpha()
    return bird
#得到背景
def load_background():
    background = pygame.image.load(background_file).convert()
    return background
#得到障碍物
def load_obstacle():
    obstacle = pygame.image.load(obstacle_file).convert()
    return obstacle

In [ ]:
# windows版本强化学习gym找金币游戏
import logging #日志模块
import numpy
import random
from gym import spaces
import gym

logging = logging.getLogger(__name__)


# Set this in SOME subclasses
class GridEnv(gym.Env):
    metadata = {
        'render.modes':['human','rgb_array'],
        'video.frames_per_second':2
    }

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def __init__(self):
        # 状态空间
        self.states = [1,2,3,4,5,6,7,8]
        # 设置机器人的位置，机器人的位置根据所处状态不同位置也不同。
        # 事先计算出每个状态点机器人位置的中心坐标，并存储到两个向量中，在类初始化中给出
        self.x = [140, 220, 300, 380, 460, 140, 300, 460]
        self.y = [250, 250, 250, 250, 250, 150, 150, 150]
        self.terminate_states = dict() #终止状态为字典格式
        self.terminate_states[6] = 1
        self.terminate_states[7] = 1
        self.terminate_states[8] = 1

        # 动作空间
        self.actions = ['n','e','s','w']

        # 回报函数
        self.rewards = dict(); #回报的数据结构为字典
        self.rewards['1_s'] = -1.0
        self.rewards['3_s'] = 1.0
        self.rewards['5_s'] = -1.0

        # 状态转移概率
        self.t = dict();
        self.t['1_s'] = 6
        self.t['1_e'] = 2
        self.t['2_w'] = 1
        self.t['2_e'] = 3
        self.t['3_s'] = 7
        self.t['3_w'] = 2
        self.t['3_e'] = 4
        self.t['4_w'] = 3
        self.t['4_e'] = 5
        self.t['5_w'] = 8
        self.t['5_w'] = 4

        #折扣因子
        self.gamma = 0.8
        self.viewer = None
        self.state = None

    def getTerminal(self):
        return self.terminate_states

    def getGamma(self):
        return self.gamma

    def getStates(self):
        return self.states

    def getAction(self):
        return self.actions

    def getTerminate_states(self):
        return self.terminate_states

    def setAction(self,s):
        self.state = s

    # step()函数输入是动作，
    # 输出是下一时刻的动作、回报、是否终止、调试信息
    # 没有的用{}表示
    def _step(self,action):
        # 系统当前状态
        state = self.state
        # 判断系统当前状态是否为终止状态
        if state in self.terminate_states:
            return state,0,True,{}
        #将状态与动作组成字典的键值
        key = "%d_%s"(state,action)

        #状态转移
        if key in self.t:
            next_state = self.t[key]
        else:
            next_state = state
        self.state = next_state

        is_terminal = False

        if next_state in self.terminate_states:
            is_terminal = True

        if key not in self.rewards:
            r = 0.0
        else:
            r = self.rewards[key]

        return next_state,r,is_terminal,{}

    # reset函数建立
    # reset常常用随机的方法初始机器人状态
    def _reset(self):
        self.state = self.states[int(random.random()*len(self.states))]
        return self.state


        # render函数建立
    def _render(self, mode='human', close=False):
        if close:
            if self.viewer is not None:
                self.viewer.close()
                self.viewer = None
            return
        screen_width = 600
        screen_height = 400

        if self.viewer is None:
            # 调用rendering的画图函数
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)

            # 创建网格世界，一共11条直线
            self.line1 = rendering.Line((100, 300), (500, 300))
            self.line2 = rendering.Line((100, 200), (500, 200))
            self.line3 = rendering.Line((100, 300), (100, 100))
            self.line4 = rendering.Line((180, 300), (180, 100))
            self.line5 = rendering.Line((260, 300), (260, 100))
            self.line6 = rendering.Line((340, 300), (340, 100))
            self.line7 = rendering.Line((420, 300), (420, 100))
            self.line8 = rendering.Line((500, 300), (500, 100))
            self.line9 = rendering.Line((100, 100), (180, 100))
            self.line10 = rendering.Line((260, 100), (340, 100))
            self.line11 = rendering.Line((420, 100), (500, 100))

            # 接下来写死亡区域，黑色实心圆代表死亡区域
            # 创建第一个骷髅
            self.kulo1 = rendering.make_circle(40)
            self.circletrans = rendering.Transform(translation=(140, 150))  # 圆心坐标
            self.kulo1.add_attr(self.circletrans)
            self.kulo1.set_color(0, 0, 0)

            # 创建第二个骷髅
            self.kulo2 = rendering.make_circle(40)
            self.circletrans = rendering.Transform(translation=(460, 150))
            self.kulo2.add_attr(self.circletrans)
            self.kulo2.set_color(0, 0, 0)

            # 创建金币区域，用浅色的圆表示
            self.gold = rendering.make_circle(40)
            self.circletrans = rendering.Transform(translation=(300, 150))
            self.gold.add_attr(self.circletrans)
            self.gold.set_color(1, 0.9, 0)

            # 创建机器人，用不同颜色的圆表示
            self.robot = rendering.make_circle(30)
            self.robotrans = rendering.Transform()
            self.robot.add_attr(self.robotrans)
            self.robot.set_color(0.8, 0.6, 0.4)

            # 给11条直线设置颜色，并将这些创建的对象添加到几何中
            self.line1.set_color(0, 0, 0)
            self.line2.set_color(0, 0, 0)
            self.line3.set_color(0, 0, 0)
            self.line4.set_color(0, 0, 0)
            self.line5.set_color(0, 0, 0)
            self.line6.set_color(0, 0, 0)
            self.line7.set_color(0, 0, 0)
            self.line8.set_color(0, 0, 0)
            self.line9.set_color(0, 0, 0)
            self.line10.set_color(0, 0, 0)
            self.line11.set_color(0, 0, 0)

            self.viewer.add_geom(self.line1)
            self.viewer.add_geom(self.line2)
            self.viewer.add_geom(self.line3)
            self.viewer.add_geom(self.line4)
            self.viewer.add_geom(self.line5)
            self.viewer.add_geom(self.line6)
            self.viewer.add_geom(self.line7)
            self.viewer.add_geom(self.line8)
            self.viewer.add_geom(self.line9)
            self.viewer.add_geom(self.line10)
            self.viewer.add_geom(self.line11)
            self.viewer.add_geom(self.kulo1)
            self.viewer.add_geom(self.kulo2)
            self.viewer.add_geom(self.gold)
            self.viewer.add_geom(self.robot)


        # 根据这两个向量和机器人当前状态，就可以设置机器人当前的圆心坐标
        if self.state is None: return None
        self.robotrans.set_translation(self.x[self.state - 1], self.y[self.state - 1])


        return self.viewer.render(return_rgb_array=mode =='rgb_array')

In [ ]:
# 基于gym构建如下迷宫世界：
import logging #日志模块
import numpy
import random
from gym import spaces
import gym

logging = logging.getLogger(__name__)


# Set this in SOME subclasses
class MazeEnv(gym.Env):
    metadata = {
        'render.modes':['human','rgb_array'],
        'video.frames_per_second':2
    }
    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]


    def __init__(self):
        # 状态空间
        self.states = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
        # 设置机器人的位置，机器人的位置根据所处状态不同位置也不同。
        # 事先计算出每个状态点机器人位置的中心坐标，并存储到两个向量中，在类初始化中给出
        self.x = [125, 175, 225, 325, 125, 175, 225, 325, 225, 275, 325, 125, 175, 225, 275, 325, 125, 175]
        self.y = [325, 325, 325, 325, 275, 275, 275, 275, 225, 225, 225, 175, 175, 175, 175, 175, 125, 125]
        self.terminate_states = dict() #终止状态为字典格式
        self.terminate_states[11] = 1

        # 动作空间
        self.actions = ['n','e','s','w']

        # 回报函数
        self.rewards = dict(); #回报的数据结构为字典
        self.rewards['18_s'] = 20.0
        self.rewards['10_e'] = 20.0
        self.rewards['16_n'] = 20.0

        # 状态转移概率
        self.t = dict();
        self.t['1_e'] = 2
        self.t['1_s'] = 5
        self.t['2_w'] = 1
        self.t['2_e'] = 3
        self.t['2_s'] = 6
        self.t['3_w'] = 2
        self.t['3_s'] = 7
        self.t['4_s'] = 8
        self.t['5_e'] = 6
        self.t['5_n'] = 1
        self.t['6_w'] = 5
        self.t['6_e'] = 7
        self.t['6_n'] = 2
        self.t['7_w'] = 6
        self.t['7_s'] = 9
        self.t['7_n'] = 3
        self.t['8_s'] = 11
        self.t['8_n'] = 4
        self.t['9_e'] = 10
        self.t['9_s'] = 14
        self.t['9_n'] = 7
        self.t['10_w'] = 9
        self.t['10_e'] = 11
        self.t['10_s'] = 15
        self.t['11_w'] = 10
        self.t['11_s'] = 16
        self.t['11_n'] = 8
        self.t['12_e'] = 13
        self.t['12_s'] = 17
        self.t['13_w'] = 12
        self.t['13_e'] = 14
        self.t['13_s'] = 18
        self.t['14_w'] = 13
        self.t['14_e'] = 15
        self.t['14_n'] = 9
        self.t['15_w'] = 14
        self.t['15_e'] = 16
        self.t['15_n'] = 10
        self.t['16_w'] = 15
        self.t['16_n'] = 11
        self.t['17_e'] = 18
        self.t['17_n'] = 12
        self.t['18_w'] = 17
        self.t['18_n'] = 13

        #折扣因子
        self.gamma = 0.8
        self.viewer = None
        self.state = None

    def getTerminal(self):
        return self.terminate_states

    def getGamma(self):
        return self.gamma

    def getStates(self):
        return self.states

    def getAction(self):
        return self.actions

    def getTerminate_states(self):
        return self.terminate_states

    def setAction(self,s):
        self.state = s

    # step()函数输入是动作，
    # 输出是下一时刻的动作、回报、是否终止、调试信息
    # 没有的用{}表示
    def _step(self,action):
        # 系统当前状态
        state = self.state
        # 判断系统当前状态是否为终止状态
        if state in self.terminate_states:
            return state,0,True,{}
        #将状态与动作组成字典的键值
        key = "%d_%s"(state,action)

        #状态转移
        if key in self.t:
            next_state = self.t[key]
        else:
            next_state = state
        self.state = next_state

        is_terminal = False

        if next_state in self.terminate_states:
            is_terminal = True

        if key not in self.rewards:
            r = -1.0
        else:
            r = self.rewards[key]

        return next_state,r,is_terminal,{}

    # reset函数建立
    # reset常常用随机的方法初始机器人状态
    def _reset(self):
        self.state = self.states[int(random.random()*len(self.states))]
        return self.state


        # render函数建立
    def _render(self, mode='human', close=False):
        if close:
            if self.viewer is not None:
                self.viewer.close()
                self.viewer = None
            return
        screen_width = 450
        screen_height = 450

        if self.viewer is None:
            # 调用rendering的画图函数
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)

            # 创建网格世界，一共12条直线
            self.line1 = rendering.Line((100, 350), (350, 350))
            self.line2 = rendering.Line((100, 300), (350, 300))
            self.line3 = rendering.Line((100, 250), (350, 250))
            self.line4 = rendering.Line((100, 200), (350, 200))
            self.line5 = rendering.Line((100, 150), (350, 150))
            self.line6 = rendering.Line((100, 100), (350, 100))
            self.line7 = rendering.Line((100, 350), (100, 100))
            self.line8 = rendering.Line((150, 350), (150, 100))
            self.line9 = rendering.Line((200, 350), (200, 100))
            self.line10 = rendering.Line((250, 350), (250, 100))
            self.line11 = rendering.Line((300, 350), (300, 100))
            self.line12 = rendering.Line((350, 350), (350, 100))

            # 接下来写墙块，黑色填满代表墙块
            # 创建第一个墙块
            self.wall1 = rendering.make_polygon([(250,350),(300,350), (300,300),(250,300)],filled=True)
            self.wall1.set_color(0, 0, 0)

            # 创建第二个墙块
            self.wall2 = rendering.make_polygon([(250,300),(300,300),(300,250),(250,250)],filled=True)
            self.wall2.set_color(0, 0, 0)

            # 创建第三个墙块
            self.wall3 = rendering.make_polygon([(100, 250), (150, 250), (150,200),(100,200)], filled=True)
            self.wall3.set_color(0, 0, 0)

            # 创建第四个墙块
            self.wall4 = rendering.make_polygon([(150, 250), (200, 250), (200,200),(150,200)], filled=True)
            self.wall4.set_color(0, 0, 0)

            # 创建第五个墙块
            self.wall5 = rendering.make_polygon([(200, 150), (250, 150), (250, 100),(200, 100)], filled=True)
            self.wall5.set_color(0, 0, 0)

            # 创建第六个墙块
            self.wall6 = rendering.make_polygon([(250, 150), (300, 150), (300, 100),(250, 100)], filled=True)
            self.wall6.set_color(0, 0, 0)

            # 创建第七个墙块
            self.wall7 = rendering.make_polygon([(300, 150), (350, 150), (350, 100),(300, 100)], filled=True)
            self.wall7.set_color(0, 0, 0)

            # 创建出口，用三角形表示
            self.goal = rendering.make_polygon([(325,245),(300,205),(350,205)],
                                               filled=True)
            self.goal.set_color(1, 0.9, 0)

            # 创建机器人，用不同颜色的圆表示
            self.robot = rendering.make_circle(15)
            self.robotrans = rendering.Transform()
            self.robot.add_attr(self.robotrans)
            self.robot.set_color(0.8, 0.6, 0.4)

            # 给11条直线设置颜色，并将这些创建的对象添加到几何中
            self.line1.set_color(0, 0, 0)
            self.line2.set_color(0, 0, 0)
            self.line3.set_color(0, 0, 0)
            self.line4.set_color(0, 0, 0)
            self.line5.set_color(0, 0, 0)
            self.line6.set_color(0, 0, 0)
            self.line7.set_color(0, 0, 0)
            self.line8.set_color(0, 0, 0)
            self.line9.set_color(0, 0, 0)
            self.line10.set_color(0, 0, 0)
            self.line11.set_color(0, 0, 0)
            self.line12.set_color(0, 0, 0)

            self.viewer.add_geom(self.line1)
            self.viewer.add_geom(self.line2)
            self.viewer.add_geom(self.line3)
            self.viewer.add_geom(self.line4)
            self.viewer.add_geom(self.line5)
            self.viewer.add_geom(self.line6)
            self.viewer.add_geom(self.line7)
            self.viewer.add_geom(self.line8)
            self.viewer.add_geom(self.line9)
            self.viewer.add_geom(self.line10)
            self.viewer.add_geom(self.line11)
            self.viewer.add_geom(self.line12)
            self.viewer.add_geom(self.wall1)
            self.viewer.add_geom(self.wall2)
            self.viewer.add_geom(self.wall3)
            self.viewer.add_geom(self.wall4)
            self.viewer.add_geom(self.wall5)
            self.viewer.add_geom(self.wall6)
            self.viewer.add_geom(self.wall7)
            self.viewer.add_geom(self.goal)
            self.viewer.add_geom(self.robot)


        # 根据这两个向量和机器人当前状态，就可以设置机器人当前的圆心坐标
        if self.state is None: return None
        self.robotrans.set_translation(self.x[self.state - 1], self.y[self.state - 1])


        return self.viewer.render(return_rgb_array=mode =='rgb_array')
